In [37]:
import os
import re
earlygreekepic = ['tlg0012','tlg0020','tlg0013']

def getepicfiles():
    rlist = []
    for (root,dirs,files) in os.walk('.'):
        m = re.search('(tlg[0-9]+)',root)
        if(not m):
            continue
        if(m and not m[1] in earlygreekepic):
            continue
        for file in files:
            if(re.search('tlg0012.tlg00[12].daphne_tb',file)):
                continue
            if(re.search('tlg0020.tlg00[123].daphne_tb',file)):
                continue
            if(re.search('\.bak',file)):
                continue
            if(re.search('conllu$',file)):
                #print(root,file)
                rlist.append(root+'/'+file)
    return(rlist)

flist = getepicfiles()
flist

['./tlg0020/tlg0020.tlg002.daphne_perstb-grc1.conllu',
 './tlg0020/tlg0020.tlg003.daphne_perstb-grc1.conllu',
 './tlg0020/tlg0020.tlg001.daphne_perstb-grc1.conllu',
 './tlg0012/tlg001/tlg0012.tlg001.daphne_perstb-grc1.1-6.conllu',
 './tlg0012/tlg001/tlg0012.tlg001.daphne_perstb-grc1.7-12.conllu',
 './tlg0012/tlg001/tlg0012.tlg001.daphne_perstb-grc1.13-18.conllu',
 './tlg0012/tlg001/tlg0012.tlg001.daphne_perstb-grc1.19-24.conllu',
 './tlg0012/tlg002/tlg0012.tlg002.daphne_perstb-grc1.7-12.conllu',
 './tlg0012/tlg002/tlg0012.tlg002.daphne_perstb-grc1.19-24.conllu',
 './tlg0012/tlg002/tlg0012.tlg002.daphne_perstb-grc1.1-6.conllu',
 './tlg0012/tlg002/tlg0012.tlg002.daphne_perstb-grc1.13-18.conllu',
 './tlg0013/tlg0013.tlg002.daphne_tb-grc1.conllu']

In [43]:
wordid2cit = {}
wordid2form= {}
wordid2lemma = {}
wordid2upos = {}
wordid2xpos = {}
wordid2feats = {}
wordid2head = {}
wordid2deprel = {}
wordid2deplist = {}
lemmaindex = {}
verbargs = {}
worddeprels = {}

def additem(curdict,curitem):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1

def add2lemmaindex(lemmaindex,lemma,wordid):
    if(not lemma in lemmaindex):
        lemmaindex[lemma] = [wordid]
    else:
        lemmaindex[lemma].append(wordid)

def loadUDfile(fname):
    cursentid = ''
    m = re.search('(tlg[0-9]+\.tlg[0-9]+)',fname)
    if(not m):
        print(fname,'expected file name with tlg[0-9]+\.tlg[0-9]+')
        return

    else:
        curtgwork = m[1]
    f = open(fname)
    for l in f:
        l = re.sub('\s+$','',l)
        if(re.search('u--------',l)):
            continue
        curcit = ''
        m = re.search('sent_id\s*=\s*([0-9a-b]+)',l)
        if(m):
            cursentid = m[1]
        m = re.search('^([0-9]+)\s',l)
        if(not m):
            continue
        args = l.split()
            
        wordid = curtgwork + ':sent' + cursentid + ':' + m[1]
        m = re.search('Ref=([^\|]+)',l)
        if(m):
            curcit = curtgwork + ':' + m[1]
        wordid2cit[wordid] = curcit
        args = l.split()
        #print(len(args),l)
        wordid2form[wordid] = args[1]
        wordid2lemma[wordid] = args[2]
        add2lemmaindex(lemmaindex,args[2],wordid)
        wordid2upos[wordid] = args[3]
        wordid2xpos[wordid] = args[4]
        wordid2feats[wordid] = args[5]
        wordid2head[wordid] = curtgwork + ':sent' + cursentid + ':' + args[6]
            
        wordid2deprel[wordid] = args[7]
    f.close()

def resolveconj():
    for wordid in wordid2deprel:
        if(wordid2lemma[wordid] == 'μῆνις'):
            print('1',wordid)
        #print('2',wordid,wordid2lemma[wordid])
        if(wordid2deprel[wordid] == 'conj'):
            wordid2deprel[wordid] = wordid2deprel[wordid2head[wordid]]
            wordid2head[wordid] = wordid2head[wordid2head[wordid]]
        if(not wordid2head[wordid] in wordid2deplist):
            wordid2deplist[wordid2head[wordid]] = [wordid]
        else:
            wordid2deplist[wordid2head[wordid]].append(wordid)
        if(not wordid in wordid2lemma):
            print('missing lemma',wordid)
            continue
        if(not wordid2lemma[wordid] in worddeprels):
            worddeprels[wordid2lemma[wordid]] = {}
        if(wordid2lemma[wordid] == 'μῆνις'):
            print(wordid,wordid2lemma[wordid],worddeprels[wordid2lemma[wordid]])
        additem(worddeprels[wordid2lemma[wordid]],wordid2deprel[wordid])

        
        


lemmadeps = {}
def getdeps(lemma):
    rlist = {}
    if(not lemma in lemmaindex):
        return

    print(lemma,len(lemmaindex[lemma]))
    for wordid in lemmaindex[lemma]:
        if(not wordid in wordid2deplist):
            continue
        print('\n'+wordid,wordid2lemma[wordid],wordid2form[wordid])
        for depid in wordid2deplist[wordid]:
            if(not lemma in verbargs):
                verbargs[lemma] = {}
            
            additem(verbargs[lemma],wordid2lemma[depid]+ ':' + wordid2deprel[depid])
            print(depid,wordid2form[depid],wordid2lemma[depid],wordid2deprel[depid])
            
for fname in flist:   
    loadUDfile(fname)
resolveconj()

curlemma = 'συνάγω'
getdeps(curlemma)
for foo in sorted(verbargs[curlemma],key=verbargs[curlemma].get,reverse=True):
    print(foo,verbargs[curlemma][foo])


1 tlg0020.tlg003:sent2273422:5
tlg0020.tlg003:sent2273422:5 μῆνις {}
1 tlg0012.tlg001:sent2274106:1
tlg0012.tlg001:sent2274106:1 μῆνις {'obj': 1}
1 tlg0012.tlg001:sent2274140:8
tlg0012.tlg001:sent2274140:8 μῆνις {'obj': 2}
1 tlg0012.tlg001:sent2275445:29
tlg0012.tlg001:sent2275445:29 μῆνις {'obj': 3}
1 tlg0012.tlg001:sent2275520:5
tlg0012.tlg001:sent2275520:5 μῆνις {'obj': 4}
1 tlg0012.tlg001:sent2275665:9
tlg0012.tlg001:sent2275665:9 μῆνις {'obj': 4, 'nsubj': 1}
1 tlg0012.tlg001:sent2277013:22
tlg0012.tlg001:sent2277013:22 μῆνις {'obj': 5, 'nsubj': 1}
1 tlg0012.tlg001:sent2278452:35
tlg0012.tlg001:sent2278452:35 μῆνις {'obj': 6, 'nsubj': 1}
1 tlg0012.tlg001:sent2278897:14
tlg0012.tlg001:sent2278897:14 μῆνις {'obj': 7, 'nsubj': 1}
1 tlg0012.tlg001:sent2279597:9
tlg0012.tlg001:sent2279597:9 μῆνις {'obj': 7, 'nsubj': 1, 'amod': 1}
1 tlg0012.tlg001:sent2280451:9
tlg0012.tlg001:sent2280451:9 μῆνις {'obj': 8, 'nsubj': 1, 'amod': 1}
1 tlg0012.tlg001:sent2280470:9
tlg0012.tlg001:sent2280470:9

In [44]:
curlem = 'συνάγω'
for foo in sorted(verbargs[curlem],key=verbargs[curlem].get,reverse=True):
    print(foo,verbargs[curlem][foo])

ἵνα:mark 2
Ἄρης:obj 2
ἔρις:obj 2
δέ:cc 2
ἄρα:advmod 2
Τρώς:nsubj 2
Δαναοί:nsubj 2
ὑσμίνη:obj 2
ἀτάρ:discourse 1
κῆρυξ:nsubj 1
ὅρκιον:obj 1
γεραιός:obj 1
ναός:obl:arg 1
αὐτός:obl 1
δή:advmod 1
εἶπον:advcl 1
νεφέλη:obj 1
ὥρα:obl 1
δε:cc 1
τῷ:advmod 1
γε:advmod 1
περιπέλομαι:advcl 1
παῖς:nsubj 1
πόλεμος:obj 1
φύλοπις:obj 1
ἀεί:advmod 1
ἦμαρ:obl 1


In [41]:
curlemma = 'χόλος'
curlemma = 'μῆνις'
curlemma = 'ἔχω'
curlemma = 'πέμπω'

print(len(lemmaindex[curlemma]))
deptot = 0
for foo in sorted(worddeprels[curlemma],key=worddeprels[curlemma].get,reverse=True):
    deptot = deptot + worddeprels[curlemma][foo]
    print(foo,worddeprels[curlemma][foo])
print(deptot,deptot/len(lemmaindex[curlemma]))

100
root 58
acl:relcl 20
ccomp 10
advcl 9
appos 1
parataxis 1
acl 1
100 1.0


In [32]:
worddeprels['χόλος']

{'obj': 40, 'nsubj': 23, 'obl': 2, 'obl:arg': 1, 'amod': 1}

In [19]:
wordid2deprel['tlg0020.tlg003:sent2273422:5']

'obj'